# CS231n Winter 2016: Lecture 8
## Topics
**TODO:** add topics. hm maybe there is some plugin to do this quick?

## Sources
- video: https://www.youtube.com/watch?v=GxZrEKZfW2o

In [1]:
from IPython.display import HTML
video_id='GxZrEKZfW2o'
HTML(f'<iframe width="560" height="315" src="https://www.youtube.com/embed/{video_id}?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

## Increase hardness
- Classification 
- Localization (one of some objects)
- Object Detection (variable number of objects)
- Instance Segmentation

## Localization as Regrassion
box (x,y,width,height)
### Regresssion head
- get pretrained model
- replace the last fully connected layer ("classification head")
- set "regression head" instead with few fully connected layers which output coors of box
- in test time we use both "heads" (classification and regression)

### Types of head
- class specific regressor or class agnostic regressor

### Design choices
- where to attach regression head
  - after conv layers - Overfeat, VGG 
  - after the last FC layer - DeepPose, R-CNN
  
### Application: Human Pose Estimation
- [DeepPose: Human Pose Estimation via Deep Neural Networks Alexander Toshev, Christian Szegedy 2014](https://arxiv.org/abs/1312.4659) uses R-CNN

### Sliding Window
- run on multiple possion and aggregate result (use heuristics)

### Application: AlexNet
- localization method was done but wasn't published

### Application: Overfeat 
- alex net class architect
  - input 3x221x221
- FC 4096 -> FC 1024 -> FC Boxes(1000x4)
- problem - expensive
- we could consider each FC layer as Conv layer
  - once we increase size of input matrix output conv layer also increases its size
  
### Application: VGG
- uses overfeat but fewer scales and localization, simpler method, but deper network

### Application: ResNet
- uses RPN and deeper features

## Object Detection
- slide different windows and try to classify (use image pyramid) 

### History
- 2005 HOG (Histogram oriented gradients)
- 2010 DPM (Deformable Parts Model)
- 2015 realize that DPM is just simple CNN [Deformable Part Models are Convolutional Neural Networks, Ross Girshick, Forrest Iandola, Trevor Darrell, Jitendra Malik](https://arxiv.org/abs/1409.5403)

### Region Proposals
- problem - a lot of tests, solution - use fast classifier but CNN is complex one, so we should decrease number of locations
- class-agnostic object detection
- it's looking for blob like stracture at an image

#### Selective Search
- [Selective Search for Object Recognition J. R. R. Uijlings](https://link.springer.com/article/10.1007/s11263-013-0620-5)
- start from pixels and merge them like regions
- put boxes around regions
- other: [What makes for effective detection proposals? Jan Hosang, Rodrigo Benenson, Piotr Dollár, Bernt Schiele](https://arxiv.org/abs/1502.05082) - Lectors fav is EdgeBoxes

### R-CNN
[Rich feature hierarchies for accurate object detection and semantic segmentation Ross Girshick, Jeff Donahue, Trevor Darrell, Jitendra Malik](https://arxiv.org/abs/1311.2524)
- apply selective search and get about 2k boxes
- preprocess: crop and warp regions
- use CNN (with regression head and classification head) for 2k boxes
- train on possitive and neg regions of image
- use multiple binarry classifiers to be able to extract multiple classes for the single region
- Quality
    - mAP (performance0) 66.0
    
#### R-CNN problems:
- slow at test-time
- use classier and regression head post-hoc - CNN features are not updated to them
- complex multistage training pipeline

### Fast R-CNN
[Fast R-CNN Ross Girshick](https://arxiv.org/abs/1504.08083)
- remove `selective search`
- apply CNN directly to the image
- get Regions of interests (RoI) - conv5 feature map of image
- add `RoI-Polling` (single-level SPP) layer - which plays role of Selective Search
- after apply FC layer
- in result 2 head: classifier (Softmax) and Boundig box-regressors (Linear)

#### Solves R-CNN problems
- faster - use single CNN for all regions (we have single region right now - the whole image)
- train all at once - sum of 2 head loss functions (Smooth L1 Loss + Log loss) - multi-task loss
- Quality
    - mAP (performance0) 66.9
    - 8.8x faster training (9.5 hours for Pascal)
    - 146x faster testing (0.32 seconds) - doesn't include region proposals
    - 25x faster testing (2 seconds) - include region proposals

### RoI-Polling
- get reg proposal
- apply `reg proposal` to Hi-res conv features: `C x H x W`
- RoI conv features: max-pool within each grid cell

### Faster R-CNN
[Faster R-CNN: Towards Real-Time Object Detection with Region Proposal Networks Shaoqing Ren, Kaiming He, Ross Girshick, Jian Sun](https://arxiv.org/abs/1506.01497)
- attach region proposal network (CNN) which creates feature map of proposals
- then (class-agnostic detector):
    - slide a small window on the convolutional feature map
    - small neural network 
      - classifying objects or non-object 
      - regressing bbox locations
    - n anchors boxes
    - classification gives probability of each anchor box
    - and coors corrections (offset)
- one network, 4 losses
  - RPN classification (anchor good/bad)
  - RPN regression (anchor -> proposal)
  - Fast R-CNN classification (over classes)
  - Fast R-CNN regression (proposal -> box)
- Quality
  - 0.2 sec, 250x 
  - mAP (performance) 66.9
- [Deep Residual Learning for Image Recognition Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun](https://arxiv.org/abs/1512.03385)

## YOLO (You Only Look Once)
[You Only Look Once: Unified, Real-Time Object Detection - Joseph Redmon, Santosh Divvala, Ross Girshick, Ali Farhadi](https://arxiv.org/abs/1506.02640)

Detection as Regression
- dive in a spatial grid (7x7)
- in each cell predict bbox and class

- Real-time detector 
  - YOLO - 45 fps, mAP=63.4
  - Fast YOLO - 155 fps, mAP=52.7

## Other
- [Spatial Transformer Networks Max Jaderberg, Karen Simonyan, Andrew Zisserman, Koray Kavukcuoglu](https://arxiv.org/abs/1506.02025) - uses bilinear interpolation and could be useful for rotation
  - another problem - we don't have that ground truth dataset 
    - but maybe we could make it if we know for sure that bound box alight correctly we just can rotate an image
- 100Hz DPM or 30Hz DPM also are real-time detectors but with mAP = 16 and mAP = 26.1

### Datasets for Object Detection
#### Pascal (VOC 2010)
- number of classes: **20**
- number of images (train + val): **~20k**
- mean objects per image: **2.4**

#### ImageNet Detection (ILSVRC 2014)
- number of classes: **200**
- number of images (train + val): **~470k**
- mean objects per image: **1.1**

#### MS-COCO (2014)
- number of classes: **80**
- number of images (train + val): **~120k**
- mean objects per image: **7.2**